In [1]:
import pandas as pd
import ClusteringAlg3 as lf
import georgeVersion as gp
import StockMarketClustering as sm

### 1a. Time Complexity of Clustering Algorithm

### Theory

If we take $n$ is the number of firms that we have in the dataset, and $k$ as the number of iterations, the algorithm follows some key properties:
* After the $k$th iteration, the maximum number of firms in a cluster is min(k+1,n), as one can simply add a link to a "new" node in every iteration, starting from k=0 where all clusters are of size 1, and one can do this up until all nodes have been connected.
* After the $k$th iteration, the minimum number of clusters is max(1,n-k), which corresponds to the case above.
* After the $k$th iteration, the maximum number of clusters is between n and n-k, in fact the maximum number of clusters is n minus the maximum number that when triangled, is less than n - specifically this is $n +  \left \lfloor{\frac{1- \sqrt{1+8k}}{2}}\right \rfloor$.

This second fact can be seen iteratively by considering the highest number of possible elements within a single cluster, building up on one maximally connected cluster will leave the remaining nodes as their own individual clusters, and therefore have the highest number of clusters.
* After the first iteration, two nodes that are not part of the same set must be merged together. (e.g. A-B). The maximal number of clusters is n-1.
* After the second iteration, again two nodes that are not part of the same set must be merged together (e.g. B-C), as the cluster contains only two nodes, whose correlation has already been "used". The maximal number of clusters is n-2.
* After the third iteration, one new node can be linked to one of the existing 2 nodes, (e.g. C-A), as the cluster is not yet complete. The maximal number of clusters is still n-2.
* For the fourth to sixth iteration, the a new node can be added in turn to the existing 3-node connected cluster (e.g. A-D, B-D, C-D). The maximal number of clusters for this group will be n-3.
* For the seventh to tenth iteration, one new node can be linked to each of the existing 4-node cluster. (e.g. A-E, B-E, C-E, D-E). The maximal number of clusters for this group will be n-4.

This pattern, (n-1, n-2, n-2, n-3, n-3, n-3, n-4, n-4, n-4, n-4...) is n minus the reversed triangle number of k, which, when using the quadratic formula backwards from k, is 

From the above, the time complexity of the clustering algorithm is as follows:

**Creating the set of firms** 
+ Constant time operation for initialising the set
+ O($n^{2}$) operations to:
    + Loop through every correlation $(n(n-1)/2)$ so O($n^{2}$)
    + Add the firm to the set O(1)
    
**Converting the firm set into a dictionary** 
+ Constant time operation to initialise the dictionary
+ n operations to create a dictionary entry per firm

**Initialising the set of start nodes**
+ Constant time to initialise the set of start nodes as the firm set (copy)

**Core Iteration: Comparing and Merging** 
+ Loop is iterated $k$ times
    + Constant time operation to pop an item from the list
    + O($k$) time to find the bottom node from the source (as k is the maximum number of elements in a cluster at time k)
    + O($k$) time to find the bottom node from the destination
    + O($k$) time if the top node needs to be found from the destination
    + Constant time to change the pointers for the required nodes, and remove an element from the set of start nodes
    
**Returning Sets** 
+ Loop is iterated per number of clusters, maximally $n$ - the reverse triangle of $k$ times, which is in the order O($n-\sqrt{k}$)
    + Inner loop is iterated per maximum number of elements in the cluster, which could be up to k
    + Constant time operation to check if the node has a pointer to another node, and to add the node to the current set
+ Constant time to add the current set to the list of sets

So overall, the time complexity is O($n^2 + k^2 + k(n-\sqrt{k})$), which can be reduced to O($n^2 + k^2$) because $k(n-\sqrt{k})$ will always be less than $kn$ which is less than max($k^2, n^2$)

### Practice

In this section we explore two alternative algorithms to the clustering algorithm, to determine the speed in practice for our dataset, this is done with multiple values for k. The algorithms in question are referred to as:
* Algorithm 1: O($n^2 + k^2$), primarily using a dictionary of pointers
* Algorithm 2: O($n^2 + k^2$), primarily using a dictionary of sets, and with a better average complexity than the above
* Algorithm 3: O($kn^2$), primarily using sets

In [38]:
firms = pd.read_csv('SP_500_firms.csv')
close_prices = pd.read_csv('SP_500_close_2015.csv')

In [39]:
dailyReturn = sm.stockReturns(close_prices)
corr, corrGraph = sm.calCorrelations(dailyReturn)

In [40]:
correlation_list = gp.correlations(dailyReturn)
ordered_list = gp.sortCorrs(correlation_list2)

In [41]:
resultssm = sm.stockClustering(corrGraph, 5000)
clusterssm = [i for i in resultssm if len(i) > 1]

In [42]:
resultsgp = gp.clusteringAlg(ordered_list, 5000)
clustersgp = [i for i in resultsgp if len(i) > 1]

In [43]:
resultslf = lf.ClusteringAlg3(ordered_list, 5000)
clusterslf = [i for i in resultslf if len(i) > 1]

With k=50, the first algorithm performs as below:

In [44]:
## Time results Algorithm 1 (LF)
%timeit resultslf = lf.ClusteringAlg3(ordered_list, 50)

10 loops, best of 3: 42.7 ms per loop


The second algorithm performs as below, which is somewhat slower:

In [45]:
## Time results Algorithm 2 (SM)
%timeit resultssm = sm.stockClustering(corrGraph, 50)

1 loop, best of 3: 179 ms per loop


The third algorithm performs as below, which is slower than the two above.

In [46]:
## Time results Algorithm 3 (GP)
%timeit resultsgp = gp.clusteringAlg(ordered_list, 5000)

1 loop, best of 3: 1.52 s per loop


With k=5000, the first algorithm performs as below:

In [47]:
## Time results Algorithm 1 (LF)
%timeit resultslf = lf.ClusteringAlg3(ordered_list, 5000)

1 loop, best of 3: 507 ms per loop


The second algorithm performs as below, which is approximately twice as quick:

In [48]:
## Time results Algorithm 2 (SM)
%timeit resultssm = sm.stockClustering(corrGraph, 5000)

1 loop, best of 3: 256 ms per loop


The third algorithm performs as below, which is slower than the two above.

In [49]:
## Time results Algorithm 3 (GP)
%timeit resultsgp = gp.clusteringAlg(ordered_list, 5000)

1 loop, best of 3: 1.53 s per loop


With k=15000, the first algorithm performs as below:

In [50]:
## Time results Algorithm 1 (LF)
%timeit resultslf = lf.ClusteringAlg3(ordered_list, 15000)

1 loop, best of 3: 1.98 s per loop


The second algorithm performs as below, which is twice as quick:

In [51]:
## Time results Algorithm 2 (SM)
%timeit resultssm = sm.stockClustering(corrGraph, 15000)

1 loop, best of 3: 527 ms per loop


The third algorithm performs as below, which is slower than algorithm 2 get quicker than algorithm 1.

In [52]:
## Time results Algorithm 3 (GP)
%timeit resultsgp = gp.clusteringAlg(ordered_list, 15000)

1 loop, best of 3: 1.71 s per loop


With k=100000, the first algorithm performs as below:

In [53]:
## Time results Algorithm 1 (LF)
%timeit resultslf = lf.ClusteringAlg3(ordered_list, 100000)

1 loop, best of 3: 14.2 s per loop


In [54]:
## Time results Algorithm 2 (SM)
%timeit resultssm = sm.stockClustering(corrGraph, 100000)

1 loop, best of 3: 2.93 s per loop


In [55]:
## Time results Algorithm 3 (GP)
%timeit resultsgp = gp.clusteringAlg(ordered_list, 100000)

1 loop, best of 3: 1.68 s per loop


This is as expected from the algorithmic complexity as algorithm 3 is linear rather than polynomial in k, and in our case k gets very large whereas n does not. In cases with a large n and a large k, the other algorithms may out perform.